In [1]:
""" Symbolically solve the system of equations to express everything in
    [H+]. Then, use the numerical solver to just find pH. This vastly reduces
    the LUT size. 
"""
import sympy as sp

# Define the symbolic variables
h, oh, hco3, co3, ca, mg, na, k, al, aloh, aloh2, aloh3, aloh4 = sp.symbols('h oh hco3 co3 ca mg na k al aloh aloh2 aloh3 aloh4')

# Define the parameters
b0, co2_atm, beta_h, beta1, beta2, beta3, beta4, beta5, kex1, kex2, kex3, kex4, kex5 = sp.symbols('b0 co2_atm beta_h beta1 beta2 beta3 beta4 beta5 kex1 kex2 kex3 kex4 kex5')
valence_Ca2, valence_Mg2, valence_Na, valence_K, valence_Al3 = sp.symbols('valence_Ca2 valence_Mg2 valence_Na valence_K valence_Al3')

# Define the equations
eq1 = sp.Eq(h * hco3 / co2_atm, 10**(-7.8136))
eq2 = sp.Eq(h * co3 / hco3, 10**(-10.3288))
eq3 = sp.Eq(h * oh, 1e-14)

eq4 = sp.Eq(h / beta_h * (beta1 / ca)**(1/valence_Ca2), kex1)
eq5 = sp.Eq(h / beta_h * (beta2 / mg)**(1/valence_Mg2), kex2)
eq6 = sp.Eq(h / beta_h * (beta3 / na)**(1/valence_Na), kex3)
eq7 = sp.Eq(h / beta_h * (beta4 / k)**(1/valence_K), kex4)
eq8 = sp.Eq(h / beta_h * (beta5 / al)**(1/valence_Al3), kex5)

eq9 = sp.Eq(aloh * h / al, 10**(-5))
eq10 = sp.Eq(aloh2 * h * h / al, 10**(-10.1))
eq11 = sp.Eq(aloh3 * h * h * h / al, 10**(-16.9))
eq12 = sp.Eq(aloh4 * h * h * h * h / al, 10**(-22.7))

eq13 = sp.Eq(h - oh - hco3 - 2*co3 + 2*ca + 2*mg + na + k + 3*al + 2*aloh + aloh2 - aloh4, b0)
#eq9 = sp.Eq(h * h * h / al, 10**(-16.9))
#eq10 = sp.Eq(hco3 * ca / h, 10**1.8487); this will cause over-determination

# Carbonate precipitation constraint
# eq10 = sp.Eq(co3 * ca, 10**(-8.48))

# h - oh - hco3 - 2*co3 + 2*ca + 2*mg + na + k + 3*al + nh4 - no3 - 2*so4 - cl = 0

# E/P | ET/P ratio => Anions become more concentrated

##########################################################################
# Solve using H+ as the basis ion: beta_h the biggest
# solving using other cations as the basis ion still result in beta_h 
#    being in the denominator sometimes. 
##########################################################################

# Solve the equations symbolically for the unknowns in terms of h
solutions = sp.solve((eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13), 
                     (oh, hco3, co3, ca, mg, na, k, al, aloh, aloh2, aloh3, aloh4))[0]

# Display the solutions
for sol in solutions:
    print(sol)

1.0e-14/h
1.53603106838503e-8*co2_atm/h
7.20443620415286e-19*co2_atm/h**2
beta1/(beta_h*kex1/h)**valence_Ca2
beta2/(beta_h*kex2/h)**valence_Mg2
beta3/(beta_h*kex3/h)**valence_Na
beta4/(beta_h*kex4/h)**valence_K
h**2*(-2.5e+36*beta2*h**2*(beta_h*kex1/h)**valence_Ca2*(beta_h*kex3/h)**valence_Na*(beta_h*kex4/h)**valence_K - 1.25e+36*beta3*h**2*(beta_h*kex1/h)**valence_Ca2*(beta_h*kex2/h)**valence_Mg2*(beta_h*kex4/h)**valence_K - 1.25e+36*beta4*h**2*(beta_h*kex1/h)**valence_Ca2*(beta_h*kex2/h)**valence_Mg2*(beta_h*kex3/h)**valence_Na + 5.0e-11*(beta_h*kex2/h)**valence_Mg2*(beta_h*kex3/h)**valence_Na*(beta_h*kex4/h)**valence_K*(2.5e+46*b0*h**2*(beta_h*kex1/h)**valence_Ca2 - 5.0e+46*beta1*h**2 + 3.84007767096258e+38*co2_atm*h*(beta_h*kex1/h)**valence_Ca2 + 3.60221810207643e+28*co2_atm*(beta_h*kex1/h)**valence_Ca2 - 2.5e+46*h**3*(beta_h*kex1/h)**valence_Ca2 + 2.5e+32*h*(beta_h*kex1/h)**valence_Ca2))/((beta_h*kex1/h)**valence_Ca2*(beta_h*kex2/h)**valence_Mg2*(beta_h*kex3/h)**valence_Na*(beta_h

In [26]:
##########################################################################
# Test solving using Ca2+ as the basis ion, but beta_h inevitably appears
# in the denominator of pH. So, this is not useful. 
##########################################################################
# Define the equations
eq1 = sp.Eq(h * hco3 / co2_atm, 10**(-7.8136))
eq2 = sp.Eq(h * co3 / hco3, 10**(-10.3288))
eq3 = sp.Eq(h * oh, 1e-14)

eq4 = sp.Eq((ca / beta1)**(1/valence_Ca2) * beta_h / h, kex1)
eq5 = sp.Eq((ca / beta1)**(1/valence_Ca2) * (beta2 / mg)**(1/valence_Mg2), kex2)
eq6 = sp.Eq((ca / beta1)**(1/valence_Ca2) * (beta3 / na)**(1/valence_Na), kex3)
eq7 = sp.Eq((ca / beta1)**(1/valence_Ca2) * (beta4 / k)**(1/valence_K), kex4)
eq8 = sp.Eq((ca / beta1)**(1/valence_Ca2) * (beta5 / al)**(1/valence_Al3), kex5)

eq9 = sp.Eq(aloh * h / al, 10**(-5))
eq10 = sp.Eq(aloh2 * h * h / al, 10**(-10.1))
eq11 = sp.Eq(aloh3 * h * h * h / al, 10**(-16.9))
eq12 = sp.Eq(aloh4 * h * h * h * h / al, 10**(-22.7))

eq13 = sp.Eq(h - oh - hco3 - 2*co3 + 2*ca + 2*mg + na + k + 3*al + 2*aloh + aloh2 - aloh4, b0)

# Solve the equations symbolically for the unknowns in terms of ca
solutions = sp.solve((eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13), 
                     (h, oh, hco3, co3, mg, na, k, al, aloh, aloh2, aloh3, aloh4))[0]

# Display the solutions
for sol in solutions:
    print(sp.simplify(sol))

beta_h*(ca/beta1)**(1/valence_Ca2)/kex1
1.0e-14*kex1/(beta_h*(ca/beta1)**(1/valence_Ca2))
1.53603106838503e-8*co2_atm*kex1/(beta_h*(ca/beta1)**(1/valence_Ca2))
7.20443620415286e-19*co2_atm*kex1**2/(beta_h**2*(ca/beta1)**(2/valence_Ca2))
beta2/(kex2/(ca/beta1)**(1/valence_Ca2))**valence_Mg2
beta3/(kex3/(ca/beta1)**(1/valence_Ca2))**valence_Na
beta4/(kex4/(ca/beta1)**(1/valence_Ca2))**valence_K
beta_h**2*(ca/beta1)**(2/valence_Ca2)*(-2.5e+36*beta2*beta_h**2*kex1*(ca/beta1)**(2/valence_Ca2)*(kex3/(ca/beta1)**(1/valence_Ca2))**valence_Na*(kex4/(ca/beta1)**(1/valence_Ca2))**valence_K - 1.25e+36*beta3*beta_h**2*kex1*(ca/beta1)**(2/valence_Ca2)*(kex2/(ca/beta1)**(1/valence_Ca2))**valence_Mg2*(kex4/(ca/beta1)**(1/valence_Ca2))**valence_K - 1.25e+36*beta4*beta_h**2*kex1*(ca/beta1)**(2/valence_Ca2)*(kex2/(ca/beta1)**(1/valence_Ca2))**valence_Mg2*(kex3/(ca/beta1)**(1/valence_Ca2))**valence_Na + (kex2/(ca/beta1)**(1/valence_Ca2))**valence_Mg2*(kex3/(ca/beta1)**(1/valence_Ca2))**valence_Na*(kex4/(c

## Test gridded lookup using known values

In [2]:
import numpy as np

valence = {'Ca2+': 2, 'Mg2+': 2, 'Na+': 1, 'K+': 1, 'Al3+': 3}
# site-specific but time-constant parameters
# layer: 1-6, in order
site = 'HBR_1'
if site == 'UC_Davis':
    co2_atm = 360.7 * 1e-6

    # net charge
    b0 = [-3.4202625501120350E-005, -3.4208144862607920E-005, -3.4168820195205782E-005, -3.5293556450653847E-005, -3.5278628081999262E-005, -2.1130570834083306E-005]

    # net charge from SO4 = 0.4 mg/L, NO3 = 0.6 mg/L, Cl- = 0.6 mg/L, NH4+ = 0.5 mg/L
    b0_rain = - 0.4e-3 / 96.0626 * 2 - 0.6e-3 / 62.0049 - 0.6e-3 / 35.4530 + 0.5e-3 / 18
    # = -7.15e-6 mol/L
    # b0 = [item*(-1) for item in b0]

    log_kex = np.array(
        [[-3.2844349800963584, -3.6249632679567445, -3.0684355419193872, -1.7581630155741577, -3.0823863173357733],
         [-3.2839239130617854, -3.6244678140943982, -3.0678740169537781, -1.7582334912804269, -3.0824959244518864],
         [-3.2875477072037307, -3.6279809007983301, -3.0718555902959714, -1.7577337731725213, -3.0817187394674921],
         [-3.4354971191033297, -3.7061104990107929, -2.8300969971540200, -1.7922756651839953, -2.9807977488723858],
         [-3.4389151358696970, -3.7088589050791878, -2.8316635602699831, -1.7917053160024419, -2.9798263732425312],
         [-3.6650316654257087, -3.9227834802788046, -3.4482911861235030, -2.1996106084487654, -3.8544587493922129]]
    )
    log_kex1 = log_kex[:, 0]
    log_kex2 = log_kex[:, 1]
    log_kex3 = log_kex[:, 2]
    log_kex4 = log_kex[:, 3]
    log_kex5 = log_kex[:, 4]

    actual_beta = np.array(
        [[0.39963567256927485, 0.12040124833583828     , 9.6243405714631011E-003, 3.7660621106624603E-002, 4.5375756919384003E-002],
         [0.39963567256927485, 0.12040124833583828     , 9.6243405714631011E-003, 3.7660621106624603E-002, 4.5375756919384003E-002],
         [0.39963567256927485, 0.12040124833583828     , 9.6243405714631011E-003, 3.7660621106624603E-002, 4.5375756919384003E-002],
         [0.25448328256607050, 9.7693428397178594E-002 , 1.7642220482230100E-002, 2.7134625241160299E-002, 9.0930432081222493E-002],
         [0.25448328256607050, 9.7693428397178594E-002 , 1.7642220482230100E-002, 2.7134625241160299E-002, 9.0930432081222493E-002],
         [0.26506441831588734, 0.13129623234272000     , 2.1212151274084996E-002, 2.9111590236425407E-002, 9.6855990588664981E-002]]
    ) # .T
else:
    co2_atm = 360.7 * 1e-6

    b0 = [0.000219894471321958, 0.000219894471321958, 0.000219894471321958, 0.000205367936823821, 0.000196015407447221, 0.000196015407447221]

    # b0 = [item*(1e2) for item in b0]

    # net charge from SO4 = 1.5 mg/L, NO3 = 1.4 mg/L, Cl- = 0.1 mg/L, NH4+ = 0.15 mg/L
    b0_rain = - 1.5e-3 / 96.0626 * 2 - 1.4e-3 / 62.0049 - 0.1e-3 / 35.4530 + 0.15e-3 / 18
    # = -4.82e-05 mol/L
    # magnitude is different, but perhaps due to negatively charged SOM?

    log_kex1 = [-1.79209505, -1.795342278, -1.749039378, -1.902450392, -1.756802863, -1.756802863]
    log_kex2 = [-1.926098828, -1.914647837, -1.846018224, -1.992002722, -1.884582637, -1.884582637]
    log_kex3 = [-1.607871139, -1.388197311, -0.816364363, -0.837629, -0.815438031, -0.815438031]
    log_kex4 = [-0.495827374, -0.530884205, -0.616390439, -1.004326625, -0.719554988, -0.719554988]
    log_kex5 = [-2.643364815, -2.418892076, -2.281595108, -2.065426511, -1.957437553, -1.957437553]

    #log_kex1 = [0.380635,0.562007,0.803846,0.923576,1.209894,1.209894]
    #log_kex2 = [0.246631,0.442701,0.706867,0.834023,1.082114,1.082114]
    #log_kex3 = [0.564859,0.969152,1.736521,1.988397,2.151259,2.151259]
    #log_kex4 = [1.676903,1.826465,1.936495,1.821699,2.247142,2.247142]
    #log_kex5 = [-0.470635,-0.061543,0.271290,0.760600,1.009259,1.009259]

    # layer = rows, cation = columns
    #actual_beta = np.array(
    #    [[0.397444098, 0.282604688, 0.230882894, 0.038555696, 0.031635278, 0.031635278], 
    #     [0.076200201, 0.057977017, 0.052495453, 0.011722675, 0.010523986, 0.010523986],
    #     [0.004607973, 0.006492062, 0.01967972 , 0.015759707, 0.014183952, 0.014183952], 
    #     [0.056254027, 0.04408549 , 0.029416617, 0.005039138, 0.004204529, 0.004204529], 
    #     [0.10453191 , 0.302178636, 0.418373869, 0.760106934, 0.812010132, 0.812010132]]
    #).T
    actual_beta = np.array(
        [[0.43060998302594150    , 8.5126188037836184E-002, 5.5971229321931564E-003, 5.4728469827586515E-002, 0.10078803243337379],
        [0.25169442296695055    , 5.3491453735693965E-002, 6.7145372122290117E-003, 3.6739693415613846E-002, 0.24382025483952172],
        [0.20298175798392826    , 4.9419387619180827E-002, 2.2099474397889129E-002, 2.3569137626352481E-002, 0.37420023417555931],
        [3.3539504255920824E-002, 1.1111036742085609E-002, 1.7103702859600750E-002, 3.6644485792170756E-003, 0.71260719980157106],
        [2.7126878424125860E-002, 9.7471022763745141E-003, 1.4631113729200012E-002, 2.5162705737175532E-003, 0.78340273762394008],
        [2.6384985505440765E-002, 9.1462404164606350E-003, 1.3903672675712474E-002, 2.3728081542617636E-003, 0.78831381479083906]]
    )

In [3]:
# from scipy.optimize import minimize
import numpy as np
import itertools as it
from tqdm import tqdm
import multiprocessing as mp


def al_charge(
    soil_ph, beta1, beta2, beta3, beta4, beta5, kex5
):
    h = 10**(-soil_ph)
    beta_h = 1 - beta1 - beta2 - beta3 - beta4 - beta5

    al = beta5/(beta_h*kex5/h)**valence['Al3+']
    aloh = 10**(soil_ph-5)*al
    aloh2 = 10**(2*soil_ph-10.1)*al
    aloh3 = 10**(3*soil_ph-16.9)*al
    aloh4 = 10**(4*soil_ph-22.7)*al

    return 2*aloh + aloh2 - aloh4


def objective_without_al(
    soil_ph,
    b0, co2_atm, beta1, beta2, beta3, beta4, beta5, kex1, kex2, kex3, kex4, kex5
):
    h = 10**(-soil_ph)
    beta_h = 1 - beta1 - beta2 - beta3 - beta4 - beta5

    oh = 10**(soil_ph-14)
    hco3 = 1.53603106838503*10**(-8+soil_ph)*co2_atm
    co3 = 7.20443620415286*10**(-19+2*soil_ph)*co2_atm
    ca = beta1/(beta_h*kex1/h)**valence['Ca2+']
    mg = beta2/(beta_h*kex2/h)**valence['Mg2+']
    na = beta3/(beta_h*kex3/h)**valence['Na+']
    k = beta4/(beta_h*kex4/h)**valence['K+']
    al = beta5/(beta_h*kex5/h)**valence['Al3+']

    charge = h - oh - hco3 - 2*co3 + 2*ca + 2*mg + na + k + 3*al

    return abs(charge - b0) / b0


def objective(
    soil_ph,
    b0, co2_atm, beta1, beta2, beta3, beta4, beta5, kex1, kex2, kex3, kex4, kex5
):
    h = 10**(-soil_ph)
    beta_h = 1 - beta1 - beta2 - beta3 - beta4 - beta5

    oh = 10**(soil_ph-14)
    hco3 = 1.53603106838503*10**(-8+soil_ph)*co2_atm
    co3 = 7.20443620415286*10**(-19+2*soil_ph)*co2_atm
    ca = beta1/(beta_h*kex1/h)**valence['Ca2+']
    mg = beta2/(beta_h*kex2/h)**valence['Mg2+']
    na = beta3/(beta_h*kex3/h)**valence['Na+']
    k = beta4/(beta_h*kex4/h)**valence['K+']
    al = beta5/(beta_h*kex5/h)**valence['Al3+']
    aloh = 10**(soil_ph-5)*al
    aloh2 = 10**(2*soil_ph-10.1)*al
    aloh3 = 10**(3*soil_ph-16.9)*al
    aloh4 = 10**(4*soil_ph-22.7)*al

    charge = h - oh - hco3 - 2*co3 + 2*ca + 2*mg + na + k + 3*al + 2*aloh + aloh2 - aloh4

    return abs(charge - b0) / b0


def find_solution(obj_func, args):
    try:
        layerid = args[-2]
        betaid = args[-1]
        args = args[:-2]

        # print(betaid)

        # use a simple grid search to find the correct pH
        grid_search = np.linspace(2, 10, 251)

        count = 0
        err = 1
        while (err > 1e-2) and (count < 100):
            obj_value = []
            for soil_ph in grid_search:
                obj_value.append(objective(soil_ph, *args))
            best = np.argmin(np.abs(obj_value))

            # the best point is not guaranteeed to be between
            # the best and the second best when the mesh is
            # too coarse! 
            # arr = np.delete(np.abs(obj_value), best)
            # second_best = np.argmin(arr)
            # if second_best >= best:
            #     second_best = second_best + 1
            # best = grid_search[best]
            # second_best = grid_search[second_best]

            best_left = grid_search[max(best-5, 0)]
            best_right = grid_search[min(best+5, len(grid_search)-1)]
            best = grid_search[best]

            # print(best)

            err = obj_func(best, *args)

            grid_search = np.linspace(best_left, best_right, 501 + int(501*err))
            count = count + 1

            # DEBUG
            # print(count, err, best_left, best_right)
        return (best, err, count, layerid, betaid)
    except:
        print(layerid, betaid)
        raise

In [10]:
# Check the outcome at the site
obs_result = np.full(6, np.nan)
err_result = np.full(6, np.nan)
count_result = np.full(6, np.nan)
for i in range(6):
    layer = i + 1
    beta_list = list(actual_beta[i, :])

    args = b0[layer-1], co2_atm, beta_list[0], beta_list[1], \
           beta_list[2], beta_list[3], beta_list[4], \
           10**log_kex1[layer-1], 10**log_kex2[layer-1], 10**log_kex3[layer-1], \
           10**log_kex4[layer-1], 10**log_kex5[layer-1]
    obs_result[i], err_result[i], count_result[i], _, _ = \
        find_solution(objective, list(args) + [i, 0])
print(obs_result)
print(count_result)

[4.39375    4.208      4.21903448 4.24       4.266375   4.26978218]
[2. 1. 2. 1. 2. 2.]


In [11]:
# Run this and run the next cell which calls balance without Al-species
# The results demonstrate that the Al-species do not affect net charge
b0_alspecies = [None] * 6
for i in range(6):
    layer = i + 1
    beta_list = list(actual_beta[i, :])
    args = beta_list[0], beta_list[1], beta_list[2], beta_list[3], \
        beta_list[4], 10**log_kex5[layer-1]
    b0_alspecies[i] = al_charge(obs_result[i], *args)
b0_alspecies

[9.11066346492931e-06,
 5.3119492260014675e-06,
 5.780454496107048e-06,
 7.24830002144941e-06,
 8.553508920178857e-06,
 8.914070670857122e-06]

In [13]:
obs_result = np.full(6, np.nan)
err_result = np.full(6, np.nan)
count_result = np.full(6, np.nan)
for i in range(6):
    layer = i + 1
    beta_list = list(actual_beta[i, :])

    args = b0[layer-1] - b0_alspecies[layer-1], co2_atm, beta_list[0], beta_list[1], \
           beta_list[2], beta_list[3], beta_list[4], \
           10**log_kex1[layer-1], 10**log_kex2[layer-1], 10**log_kex3[layer-1], \
           10**log_kex4[layer-1], 10**log_kex5[layer-1]

    obs_result[i], err_result[i], count_result[i], _, _ = \
        find_solution(objective_without_al, list(args) + [i, 0])
print(obs_result)
print(count_result)

[4.40397605 4.208      4.22552566 4.24       4.27672118 4.2802361 ]
[100.   1. 100.   1. 100. 100.]


In [30]:
# Check the outcome at any beta values
b0 = 2.1792976671467456E-004
co2_atm = 3.3390574067422823E-004
# beta_list = [0.41469274923369581, 8.2624181239314767E-002, 6.4680556069172997E-003, 6.0696410534494954E-002, 0.11400476272795355]
beta_list = [2.2003011543048931E-003, 3.1315995871589793E-004, 2.0000000000000001E-004, 2.7503959845223691E-004, 2.0000000000000001E-004]
keq_list = [1.2702667041324166E-003, 5.6193813937540336E-004, 2.4199165578020466E-003, 1.4710196291089925E-002, 7.0704212198109207E-004]

args = b0, co2_atm, beta_list[0], beta_list[1], \
  beta_list[2], beta_list[3], beta_list[4], \
  keq_list[0], keq_list[1], keq_list[2], keq_list[3], keq_list[4]
obs_result, err, count, _, _ = find_solution(list(args) + [0, 0])
obs_result, err, count, objective(obs_result, *args)

(3.9000000000000004, 0.002453768641744377, 1, 0.002453768641744377)